In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time




In [2]:
# Load the dataframe with the YouTube video links and titles
df = pd.read_csv('comm173_youtube.csv')



In [3]:
print(df)

                                                  link  \
0    https://www.youtube.com/watch?v=3qTZnwRbFY0&pp...   
1    https://www.youtube.com/watch?v=L_2jL8vNiNM&pp...   
2    https://www.youtube.com/watch?v=gXjcwnEpILg&pp...   
3    https://www.youtube.com/watch?v=jBFMDZ5ugd4&pp...   
4    https://www.youtube.com/watch?v=3XDcNI31T2k&pp...   
..                                                 ...   
565  https://www.youtube.com/watch?v=9214acpnJcs&pp...   
566  https://www.youtube.com/watch?v=lEAewKPzZQk&pp...   
567  https://www.youtube.com/watch?v=IZqcEk9Seew&pp...   
568  https://www.youtube.com/watch?v=_LhVDt-qG-8&pp...   
569  https://www.youtube.com/watch?v=IyWSnvgIs6s&pp...   

                                                 title  description  \
0    This Is Why Costco Only Charges $5 For A Rotis...          NaN   
1    The Big Problem Costco Shoppers Are Currently ...          NaN   
2    Don't Eat Another Costco Rotisserie Chicken Un...          NaN   
3       Removing Me

In [4]:
df[~df.link.str.contains("/shorts")]

,link,title,description,category
0,https://www.youtube.com/watch?v=3qTZnwRbFY0&pp...,This Is Why Costco Only Charges $5 For A Rotis...,NaN,costco_chicken
1,https://www.youtube.com/watch?v=L_2jL8vNiNM&pp...,The Big Problem Costco Shoppers Are Currently ...,NaN,costco_chicken
2,https://www.youtube.com/watch?v=gXjcwnEpILg&pp...,Don't Eat Another Costco Rotisserie Chicken Un...,NaN,costco_chicken
3,https://www.youtube.com/watch?v=jBFMDZ5ugd4&pp...,Removing Meat from a Costco Rotisserie Chicken,NaN,costco_chicken
4,https://www.youtube.com/watch?v=3XDcNI31T2k&pp...,10 Easy Costco Chicken Dinners | Recipes You C...,NaN,costco_chicken
...,...,...,...,...
565,https://www.youtube.com/watch?v=9214acpnJcs&pp...,Fight hunger and support local artists at Empt...,NaN,costco_chicken
566,https://www.youtube.com/watch?v=lEAewKPzZQk&pp...,I review viewers' skirts. (...and UruMaid appe...,NaN,costco_chicken
567,https://www.youtube.com/watch?v=IZqcEk9Seew&pp...,ALL YOU CAN EAT CRAB SEAFOOD HOT POT & BBQ BUF...,NaN,costco_chicken
568,https://www.youtube.com/watch?v=_LhVDt-qG-8&pp...,"Culver's Drive Thru, Pot Roast Sandwich Meal, ...",NaN,costco_chicken


In [5]:
df_2 = df[~df.link.str.contains("/shorts")]

In [6]:
# Set up the Selenium webdriver
driver = webdriver.Chrome(('/Users/adityaranade/Documents/COMM173oldlaptop/chromedriver'))



/var/folders/f_/txmdpv393xx063w86f4glcx40000gn/T/ipykernel_11582/202429175.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(('/Users/adityaranade/Documents/COMM173oldlaptop/chromedriver'))


In [7]:
driver.maximize_window() # For maximizing window
driver.implicitly_wait(20) # gives an implicit wait for 20 seconds

In [8]:
# Create an empty list to hold the comments and comment likes:
comments = []
comment_likes = []

In [9]:
def scroll_to_bottom(driver):
    old_position = 0
    new_position = None
    scroll_distance = 500
    
    while (new_position != old_position):
        # Get old scroll position
        old_position = driver.execute_script(("return (window.pageYOffset !== undefined) ?"
                                            " window.pageYOffset : (document.documentElement ||"
                                            " document.body.parentNode || document.body);"))
        # Sleep and scroll
        time.sleep(5)
        driver.execute_script(("var scrollingElement = (document.scrollingElement ||"
                              " document.body);scrollingElement.scrollTop = "
                              " scrollingElement.scrollHeight"))
        
        time.sleep(5)
        # Get new position
        new_position = driver.execute_script(("return (window.pageYOffset !== undefined) ?"
                                             " window.pageYOffset : (document.documentElement ||"
                                             " document.body.parentNode || documentBody);"))

In [10]:
# Loop through the rows in the dataframe
for index, row in df_2.iterrows():
    # Navigate to the YouTube video page
    driver.get(row['link'])

    # Wait for the comments to load
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#comments #content-text')))
    
    scroll_to_bottom(driver)
    
    time.sleep(2)

    # Get the comment and comment like elements and extract their text
    comment_elems = driver.find_elements(By.CSS_SELECTOR,'#comments #content-text')
    comment_like_elems = driver.find_elements(By.CSS_SELECTOR,'#comments #vote-count-middle')
    comment_text = [elem.text for elem in comment_elems]
    comment_like_text = [elem.text for elem in comment_like_elems]

    # Append the comment and comment like text to the corresponding lists
    for i in range(len(comment_text)):
        comments.append([row['link'], row['title'], comment_text[i]])
        comment_likes.append([row['link'], row['title'], comment_text[i], comment_like_text[i]])

KeyboardInterrupt: 

In [11]:
# Close the browser window
driver.quit()


In [12]:
# Create dataframes for the comments and comment likes
comments_df = pd.DataFrame(comments, columns=['Link', 'Title', 'Comment'])
comment_likes_df = pd.DataFrame(comment_likes, columns=['Link', 'Title', 'Comment', 'Likes'])


In [13]:
# Save the dataframes to CSV files
comments_df.to_csv('video_comments.csv', index=False)
comment_likes_df.to_csv('video_comment_likes.csv', index=False)